# Composite "Closest Date"
Takes the last image of the given collection and fills the masked pixels with the last available pixel

In [1]:
import ee
ee.Initialize()

In [3]:
from geetools import tools, composite, cloud_mask, algorithms

In [4]:
import geedatasets
import ipygee as ui

In [5]:
Map = ui.Map()
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [6]:
l8 = geedatasets.landsat.Landsat8SR()

In [7]:
vis = l8.visualization('NSR')

In [9]:
l8col = l8.collection()

In [10]:
p = ee.Geometry.Point([-70.72, -41.92]).buffer(50000)

In [11]:
Map.addLayer(p)

In [12]:
Map.centerObject(p, zoom=8)

### add date band

In [13]:
col = l8col.map(lambda img: img.addBands(tools.date.makeDateBand(img)))

### Filter and mask clouds

In [14]:
col = col.filterBounds(p)\
         .filterDate('2017-01-01', '2017-03-01')\
         .map(lambda img: l8.applyMask(img, 'pixel_qa', ['cloud', 'shadow', 'snow']))

In [15]:
ui.eprint(col.size())

### add all images to the map

In [16]:
Map.addImageCollection(col, vis, '{system_date} {id}')

## Make composite

In [17]:
comp = composite.closestDate(col)

In [18]:
Map.addLayer(comp, vis, 'composite')

In [19]:
Map.addLayer(comp.select('date').randomVisualizer(), {'bands':['viz-red', 'viz-green', 'viz-blue'], 'min':0, 'max':255}, 'dates')

### clip to first image (target date)

In [20]:
comp_clip = composite.closestDate(col, clip_to_first=True)

In [21]:
Map.addLayer(comp_clip, vis, 'composite 2017-02-25 clip to first')